In [1]:
from preprocess import * 

############## ONLY NEED TO CHANGE THE FOLLOWING TO USE preprocess module##########
DIR = 'data' 
BOOK_FILE = '{}/book.txt'.format(DIR)
BOOK_LEMMA_FILE = '{}/lemmatize_book.txt'.format(DIR)
PPVT_LEMMA_FILE = '{}/PPVT lemma.csv'.format(DIR)

## Preprocessing

In [2]:
ranked_words= read_rebecca_lemma(PPVT_LEMMA_FILE)
book_words, pos_words = lemmatize_book(BOOK_FILE, BOOK_LEMMA_FILE)
generate_wordset_files(book_words, ranked_words, DIR)

85
4751
228


## Training
    - Good to see increase in distance for these 3 pairs of examples

In [3]:
import gensim
from collections import defaultdict
from scipy import spatial
from gensim.models import KeyedVectors
from igraph import *
import random
import pandas as pd
from collections import Counter
"""
task
1. `model.py`: construct simple pipeline to build igraph, form df
2. `preprocess.py`: build several graphs (V,N) in ipython book
3. `visualize.py`: visualize network by sampling
"""

DIR = 'data'
# paths to the 3 generated files from preprocess.py
BOOK_WORD_SET_FILE = '{}/book_words_set.txt'.format(DIR)
REBECCA_WORD_FILE = '{}/rebecca_words.txt'.format(DIR)
LEMMA_BOOK_FILE = "{}/lemmatize_book.txt".format(DIR)

def read_file_to_list(filename):
	words = []
	with open(filename, 'r') as f:
	    for word in f:
	        words.append(word.strip())
	return words

def getSimilarity(w1, w2, model):
    return abs(1-spatial.distance.cosine(model.wv[w1], model.wv[w2]))

def load_pretrained_word2vec(documents, size, min_count, iters, retrain=False):
	GOOGLE_W2V_FILE = './model/GoogleNews-vectors-negative300.bin'
	model = KeyedVectors.load_word2vec_format(GOOGLE_W2V_FILE, binary=True)  
	if not retrain:
		return model
	model_2 = gensim.models.Word2Vec(
	            documents,
	            sg=0,
	            size=size,
	            window=20,
	            min_count=min_count,
	            iter=iters,
	            workers=4)
	total_examples = model_2.corpus_count
	model_2.build_vocab([list(model.vocab.keys())], update=True)
	model_2.intersect_word2vec_format(GOOGLE_W2V_FILE, binary=True, lockf=1.0)
	model_2.train(documents, total_examples=total_examples, epochs=model_2.epochs)
	return model_2

def train_word2vec(documents, size=300, min_count=5, iters=100, window=20):
	model = gensim.models.Word2Vec(
        documents,
        sg=0,
        size=size,
        window=window,
        min_count=min_count,
        iter=iters,
        workers=4)
	model.train(documents, total_examples=len(documents), epochs=model.epochs)
	return model

def read_docs():
	with open(LEMMA_BOOK_FILE) as f:
	    documents = []
	    for line in f:
	        documents.append(line.split())
	return documents

def freq_counts(documents):
    allwords = []
    for doc in documents:
        allwords += doc
    
    ct = Counter(allwords)
    for i in range(5):
        print('freq ', i+1, len([w for w in ct if ct[w] >= i+1]))

In [4]:
documents = read_docs()
model = train_word2vec(documents, size=100, min_count=0, iters=40, window=10)

In [5]:
freq_counts(documents)
print('table', 'girl', getSimilarity('table', 'girl', model))
print('man', 'woman', getSimilarity('man', 'woman', model))
print('girl', 'boy', getSimilarity('girl', 'boy', model))
print('sun', 'moon', getSimilarity('sun', 'moon', model))
print('cat', 'dog', getSimilarity('cat', 'dog', model))

freq  1 10183
freq  2 2828
freq  3 1809
freq  4 1454
freq  5 1224
table girl 0.07248084247112274
man woman 0.42470675706863403
girl boy 0.6075921654701233
sun moon 0.7933577299118042
cat dog 0.8389257788658142


In [142]:
def build_graph(model, sampling_size=0):

	book_set = read_file_to_list(BOOK_WORD_SET_FILE)
	edu_set = read_file_to_list(REBECCA_WORD_FILE)

	print("word2vec:\t", len(model.wv.vocab))
	print("book_set:\t", len(book_set))
	print("edu_set:\t", len(edu_set))
# 	surrounding_words = set(model.wv.vocab.keys()).intersection(book_set)
# 	surrounding_words = set(surrounding_words).difference(edu_set)
# 	print('surrounding words', len(surrounding_words))
	intersection_words = set(model.wv.vocab.keys()).intersection(edu_set)
	print('intersertion words', len(intersection_words))

	if sampling_size:
		token2vec = {}
		rand_words = random.sample(intersection_words, sampling_size)
		for word in rand_words:
			token2vec[word] = model.wv[word]
		for word in intersection_words:
			token2vec[word] = model.wv[word]
	else:
		token2vec = model.wv.vocab # bad

	idx = 0
	id2token = {}
	token2id = {}

	for word in token2vec:
	    id2token[idx] = word
	    token2id[word] = idx
	    idx += 1

	vertices = [idx for idx in range(len(token2vec))]
	edges = [(i, j) for i in vertices for j in vertices if i < j]
	g = Graph(vertex_attrs={"label":vertices}, edges=edges, directed=False)
	g.es["sim"] = [getSimilarity(id2token[i], id2token[j], model) for i,j in edges]
	g.es["dist"] = np.array(1-np.array(g.es['sim'])).tolist()
	# test validity
	# assert(getSimilarity('man', 'woman', model) == g[token2id['man'], token2id['woman']] )
	# assert(getSimilarity('cat', 'dog', model) == g[token2id['cat'], token2id['dog']] )

	return g, (edges, vertices), (id2token, token2id)

def filter_graph(weights, edges, vertices, id2token,  threshold=1.0):

	new_edges, new_weights = [], [] # weights: similarity
	new_distances = []
	for edge, w in zip(edges, weights):
	    if w >= threshold:
	        new_edges.append(edge)
	        new_weights.append(w)
	        new_distances.append(1-w)
	g0 = Graph(vertex_attrs={"label":vertices}, edges=new_edges, directed=False)

	g0.es["sim"] = new_weights
	g0.es["dist"] = new_distances
	g0.vs["label"] = [id2token[idx] for idx in vertices]

	return g0

def compute_measures_df(g, edges, vertices, id2token, token2id):
	"""
	compute centrality measures to output df
	"""

	# g -> strength, closeness (continuous)
	strengthRank = g.strength(None,  weights=g.es['sim'])
	closenessRank = g.closeness(None, 'all', weights=g.es['dist'], normalized=True)

	# g1 -> betweenness, eigen_centrality
	g1 = filter_graph(g.es["sim"], edges, vertices, id2token, threshold=0.5)
    
	eigen_centralityRank = g.eigenvector_centrality(directed=False, weights=g.es['sim'])

	betweennessRank = g.betweenness(vertices=None, 
                                     directed=False, 
                                     weights=g.es['dist'])

	# g2 -> degree
	g2 = filter_graph(g.es["sim"], edges, vertices, id2token, threshold=0.2)

	degreeRank = g2.degree(None, mode='all')
	print(degreeRank)

	# frequency
	wordCounter = defaultdict(int)
	with open(LEMMA_BOOK_FILE) as f:
	    for line in f:
	        for w in line.split():
	            w = w.strip()
	            if w in token2id:
	                wordCounter[w] += 1
	freqRank = [wordCounter[w] for w in token2id]

	# rebecca
	with open(REBECCA_WORD_FILE, 'r') as f:
	    edu_list = []
	    for word in f:
	        edu_list.append(word.strip())
	        
	rebeccaRank = {w:i+1 for i, w in enumerate(edu_list)}
	final_words_set = set(token2id.keys()).intersection(edu_list)

	print(len(strengthRank))
	print(len(betweennessRank))
	print(len(closenessRank))
	print(len(eigen_centralityRank))
	print(len(degreeRank))

	# create df
	data = []
	words_inorder = [id2token[idx] for idx in range(len(token2id))]
	for i, word in enumerate(words_inorder):
	    if word in final_words_set:
	        data.append([word,
	                    rebeccaRank[word],
	                    strengthRank[i], 
	                    closenessRank[i], 
	                    betweennessRank[i], 
	                    eigen_centralityRank[i],
	                    degreeRank[i],
	                    freqRank[i]])


	df = pd.DataFrame(data, columns=['word', 'ppvt', 'strgth', 
                                     'close', 'betw', 'eigen', 'degree', 'freq'])
	df = df.sort_values(by=['ppvt'])
	print(df.head())
	return df

def build_pos_graph(model, pos_word_set, sampling_size=0, max_dist=1):
	"""
	build semantic graph without referring Rebecca's word list;
	only to explore noun, verb's centrality in vocab
	"""

	book_set = read_file_to_list(BOOK_WORD_SET_FILE)
	edu_set = read_file_to_list(REBECCA_WORD_FILE)
	valid_pos_words = set(model.wv.vocab.keys()).intersection(edu_set)
	valid_pos_words = set(valid_pos_words).intersection(pos_word_set)
	# print('pos_word_set', len(pos_word_set))

	token2vec = {}
	if sampling_size > 0:
		rand_words = random.sample(valid_pos_words, sampling_size)
		for word in rand_words:
			token2vec[word] = model.wv[word]			
	else:
		token2vec = valid_pos_words

	idx = 0
	id2token = {}
	token2id = {}

	for word in token2vec:
	    id2token[idx] = word
	    token2id[word] = idx
	    idx += 1

	vertices = [idx for idx in range(len(token2vec))]
	edges = [(i, j) for i in vertices for j in vertices if i < j]
	weights = [getSimilarity(id2token[i], id2token[j], model) for i, j in edges]

	g = filter_graph(weights, edges, vertices, id2token, max_dist)

	return g, (edges, vertices), (id2token, token2id)


In [143]:
# build graph     
g, (edges, vertices), (id2token, token2id) = build_graph(model, sampling_size=30)

# compute measures
df = compute_measures_df(g, edges, vertices, id2token, token2id)

# see correlations
for col in df.columns[2:]:  
    print('rebecca vs. {}'.format(col), df['ppvt'].corr(df[col].rank(ascending=False)))

word2vec:	 10183
book_set:	 4751
edu_set:	 228
intersertion words 75
[4, 7, 3, 1, 4, 30, 9, 2, 34, 3, 2, 1, 9, 31, 26, 33, 31, 14, 5, 1, 11, 3, 0, 31, 5, 2, 13, 3, 29, 30, 29, 33, 34, 1, 12, 21, 14, 26, 6, 24, 3, 26, 28, 0, 1, 30, 33, 3, 32, 27, 30, 30, 3, 31, 0, 9, 6, 2, 1, 2, 26, 2, 2, 31, 2, 4, 15, 4, 33, 22, 7, 30, 12, 3, 1]
75
75
75
75
75
     word  ppvt     strgth     close  betw     eigen  degree  freq
67   ball     1   5.404695  1.078791   0.0  0.133588       4    10
71    dog     2  18.106690  1.350406   0.0  0.759688      30    66
49  spoon     3  18.023623  1.358179   6.0  0.749242      27     7
42   foot     4  20.275800  1.405886  14.0  0.861448      28    29
15   duck     5  19.830091  1.388278   1.0  0.819107      33    17
rebecca vs. strgth 0.3474588410248375
rebecca vs. close 0.348831111329043
rebecca vs. betw 0.2850541128777261
rebecca vs. eigen 0.3743792045577737
rebecca vs. degree 0.35423651603868406
rebecca vs. freq 0.5152850749928397


In [131]:
for mname in ['strgth', 'close', 'betw', 'eigen', 'degree', 'freq']:
    cname = mname + '-rank'
    df[cname] = df[mname].rank(ascending=False)

In [132]:
df.head()

,word,ppvt,strgth,close,betw,eigen,degree,freq,strgth-rank,close-rank,betw-rank,eigen-rank,degree-rank,freq-rank
69,ball,1,5.404695,1.078791,0.0,0.133588,0,10,62.0,62.0,48.0,59.0,54.0,17.5
73,dog,2,18.106690,1.350406,0.0,0.759688,19,66,18.0,19.0,48.0,16.0,15.0,2.0
53,spoon,3,18.023623,1.358179,6.0,0.749242,15,7,19.0,17.0,11.0,19.0,21.0,23.5
45,foot,4,20.275800,1.405886,14.0,0.861448,19,29,11.0,11.0,8.0,10.0,15.0,5.0
58,duck,5,19.830091,1.388278,1.0,0.819107,20,17,12.0,13.0,19.5,12.0,12.0,9.0


In [ ]:
pmodel = load_pretrained_word2vec(documents, 100, 0, 40, retrain=False)

In [65]:
freq_counts(documents)
print('table', 'girl', getSimilarity('table', 'girl', pmodel))
print('man', 'woman', getSimilarity('man', 'woman', pmodel))
print('girl', 'boy', getSimilarity('girl', 'boy', pmodel))
print('sun', 'moon', getSimilarity('sun', 'moon', pmodel))
print('cat', 'dog', getSimilarity('cat', 'dog', pmodel))

freq  1 10183
freq  2 2828
freq  3 1809
freq  4 1454
freq  5 1224
table girl 0.03306927904486656
man woman 0.7664012312889099
girl boy 0.8543271422386169
sun moon 0.42628341913223267
cat dog 0.760945737361908


/Users/hang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [28]:
from sklearn.metrics.pairwise import cosine_similarity

from numpy import dot
from numpy.linalg import norm
a = model.wv['cat']
b = model.wv['dog']
cos_sim = dot(a, b)/(norm(a)*norm(b))
cos_sim

0.43008167

## Overlapping in Vocab
    - therefore, we do not use google pre-trained word embeddings

In [133]:
vocab_overlapping_googleW2V = len([w for w in ranked_words if w in set(model.wv.vocab)])
print("Intersection b/w Google W2V, FB data, and PPVT is {}".format(vocab_overlapping_googleW2V))

book_set = set(book_words)
vocab_overlapping_fb_data = len([w for w in ranked_words if w in book_set])
print("Intersection b/w FB data, and PPVT is {}".format(vocab_overlapping_fb_data))

Intersection b/w Google W2V, FB data, and PPVT is 75
Intersection b/w FB data, and PPVT is 85


## Visualization
    - nouns
    - verbs

In [134]:
def normalize_list(l, lower, upper):
	max_min_range = max(l)-min(l)
# 	print('times', (upper-lower)/max_min_range)
	return [lower + (upper - lower) / max_min_range * x for x in l]

for i in range(0, 20, 1):
	max_dist = 0.01*(i+1)
	g, (edges, vertices), (id2token, token2id) = build_pos_graph(model, pos_words['n'], sampling_size=10, max_dist=max_dist)
	wordCounter = defaultdict(int)
	with open(LEMMA_BOOK_FILE) as f:
	    for line in f:
	        for w in line.split():
	            w = w.strip()
	            if w in token2id:
	                wordCounter[w] += 1
	rank = [wordCounter[w] for w in token2id]
	vertexSize = normalize_list(rank, 10, 40)
# 	vertexLabelSize = normalize_list(rank, 20, 40)
	visual_style = {}
	visual_style["vertex_color"] = "#94a1b6"
	visual_style["vertex_label_color"] = "#f20606"
	visual_style["vertex_label_size"] = 15 # 15
	visual_style["vertex_size"] = vertexSize # 15
	visual_style["vertex_label"] = g.vs["label"]
	layout = g.layout("kk")
	visual_style["layout"] = layout
	visual_style["bbox"] = (300, 300)
	visual_style["margin"] = 20
	filename = "./output/noun{}.pdf".format(max_dist)
	plot(g, filename, **visual_style)

In [135]:
def normalize_list(l, lower, upper):
	max_min_range = max(l)-min(l)
# 	print('times', (upper-lower)/max_min_range)
	return [lower + (upper - lower) / max_min_range * x for x in l]

for i in range(0, 20, 1):
	max_dist = 0.01*(i+1)
	g, (edges, vertices), (id2token, token2id) = build_pos_graph(model, pos_words['v'], sampling_size=10, max_dist=max_dist)
	wordCounter = defaultdict(int)
	with open(LEMMA_BOOK_FILE) as f:
	    for line in f:
	        for w in line.split():
	            w = w.strip()
	            if w in token2id:
	                wordCounter[w] += 1
	rank = [wordCounter[w] for w in token2id]
	vertexSize = normalize_list(rank, 10, 40)
# 	vertexLabelSize = normalize_list(rank, 20, 40)
	visual_style = {}
	visual_style["vertex_color"] = "#94a1b6"
	visual_style["vertex_label_color"] = "#f20606"
	visual_style["vertex_label_size"] = 15 # 15
	visual_style["vertex_size"] = vertexSize # 15
	visual_style["vertex_label"] = g.vs["label"]
	layout = g.layout("kk")
	visual_style["layout"] = layout
	visual_style["bbox"] = (300, 300)
	visual_style["margin"] = 20
	filename = "./output/noun{}.pdf".format(max_dist)
	plot(g, filename, **visual_style)

## rank words

In [145]:
# from model import *

# documents = read_docs()

# # no pretrained
# model = train_word2vec(documents, 300, 5, 200)
# # use pretrained model
# # model = load_pretrained_word2vec(documents, 300, 5, 100, retrain=False)
# # use pretrained + retrain
# # model = load_pretrained_word2vec(documents, 300, 5, 30, retrain=True)

# freq_counts(documents)
# print('man', 'woman', getSimilarity('man', 'woman', model))
# print('cat', 'dog', getSimilarity('cat', 'dog', model))
# print('sun', 'kid', getSimilarity('sun', 'kid', model))        
        
# build graph     
g, (edges, vertices), (id2token, token2id) = build_graph(model, sampling_size=10)

# compute measures
df = compute_measures_df(g, edges, vertices, id2token, token2id)

# see correlations
for col in df.columns[2:]:  
    print('rebecca vs. {}'.format(col), df['ppvt'].corr(df[col].rank(ascending=False)))
    
for mname in ['strgth', 'close', 'betw', 'eigen', 'degree', 'freq']:
    cname = mname + '-rank'
    df[cname] = df[mname].rank(ascending=False)

# save to csv
df.to_csv("./output/rankings_0625_old_book.csv",index=False)

word2vec:	 10183
book_set:	 4751
edu_set:	 228
intersertion words 75
[2, 30, 22, 2, 3, 14, 3, 33, 3, 30, 29, 30, 34, 9, 1, 12, 29, 5, 21, 14, 26, 6, 4, 24, 3, 3, 31, 26, 26, 34, 2, 28, 0, 0, 1, 33, 3, 32, 9, 27, 30, 30, 3, 13, 33, 31, 0, 9, 31, 6, 31, 1, 2, 4, 1, 1, 26, 2, 11, 2, 31, 2, 1, 4, 15, 4, 33, 5, 7, 7, 30, 12, 3, 1, 2]
75
75
75
75
75
     word  ppvt     strgth     close  betw     eigen  degree  freq
65   ball     1   5.404695  1.078791   0.0  0.133588       4    10
70    dog     2  18.106690  1.350406   0.0  0.759688      30    66
39  spoon     3  18.023623  1.358179   6.0  0.749242      27     7
31   foot     4  20.275800  1.405886  14.0  0.861448      28    29
44   duck     5  19.830091  1.388278   1.0  0.819107      33    17
rebecca vs. strgth 0.3474588410248375
rebecca vs. close 0.348831111329043
rebecca vs. betw 0.2850541128777261
rebecca vs. eigen 0.3743792045577737
rebecca vs. degree 0.35423651603868406
rebecca vs. freq 0.5152850749928397
